In [1]:
import requests
import json
import pandas as pd
from utility import date_to_timestamp,epoch_to_timestamp, timestamp_to_epoch
def get_validator_income_detail_history(validator_index, latest_epoch):
    """
    get the income detail history of a validator for the previous 100 epochs by using the beaconcha.in API
    """
    url = f'https://beaconcha.in/api/v1/validator/{validator_index}/incomedetailhistory?latest_epoch={latest_epoch}'
    response = requests.get(url)
    if response.status_code == 200:
        return json.loads(response.text)
    else:
        return None
def process_income_detail_history(res):
    """
    Porcess the income detail history of a validator
    """
    tt1=pd.DataFrame(res['data'])
    tt1['attestation_source_reward'] = tt1['income'].apply(lambda x: x['attestation_source_reward'] if 'attestation_source_reward' in x else 0)
    tt1['attestation_target_reward'] = tt1['income'].apply(lambda x: x['attestation_target_reward'] if 'attestation_target_reward' in x else 0)
    tt1['attestation_head_reward'] = tt1['income'].apply(lambda x: x['attestation_head_reward'] if 'attestation_head_reward' in x else 0)
    tt1['other_reward'] = tt1['income'].apply(lambda x: sum([int(x[i]) if i not in ['attestation_source_reward','attestation_target_reward','attestation_head_reward'] and 'penalty' not in i else 0 for i in x.keys()]))
    tt1['penalty'] = tt1['income'].apply(lambda x: sum([int(x[i]) if 'penalty' in i else 0 for i in x.keys()]))
    return tt1


In [2]:
def get_one_day_reward_one_validator(validator_index,date):
    #Get the earliest timestamp for the given date, where the date is a string '2022-10-30', and the earliest timestamp is '2022-10-30 00:00:00'.
    start_timestamp=date_to_timestamp(date)
    #Get the latest timestamp of the day for the given date string '2022-10-30', which is 2022-10-30 23:59:59.
    end_timestamp=date_to_timestamp(date,latest=True)
    start_epoch=int(timestamp_to_epoch(start_timestamp)+1)
    end_epoch=int(timestamp_to_epoch(end_timestamp))
    print(start_epoch,end_epoch)
    validator_rewards_total=pd.DataFrame()
    for epoch in range(end_epoch,start_epoch-100,-100):
        income_detail_history=get_validator_income_detail_history(validator_index,epoch)
        if income_detail_history and start_epoch-epoch<100:
            temp=process_income_detail_history(income_detail_history)
            validator_rewards_total=pd.concat([validator_rewards_total,temp])
        else:
            print('no data')
            break
    validator_rewards_total.drop(columns=['income','week','week_start','week_end'], inplace=True)  
    validator_rewards_total=validator_rewards_total.drop_duplicates()
    validator_rewards_total['time']=validator_rewards_total['epoch'].map(epoch_to_timestamp)
    validator_rewards_total['time']=pd.to_datetime(validator_rewards_total['time'],unit='s')
    validator_rewards_total['date']=validator_rewards_total['time'].dt.date
    validator_rewards_total=validator_rewards_total.sort_values(by=['epoch'])
    validator_rewards_total=validator_rewards_total[(validator_rewards_total['date']==pd.to_datetime(date).date())]
    validator_rewards_total['total_attestation_reward']=validator_rewards_total['attestation_source_reward']+validator_rewards_total['attestation_target_reward']+validator_rewards_total['attestation_head_reward']
    validator_rewards_total['total_reward']=validator_rewards_total['total_attestation_reward']+validator_rewards_total['other_reward']-validator_rewards_total['penalty']
    return validator_rewards_total   
# get_one_day_reward_one_validator(123356,'2022-10-30')

In [3]:
import random
#randomly select 10 validator in ramdon 10 days
validator_index_list=random.sample(range(0, 100000), 10)
date_list=random.sample(pd.date_range('2022-09-15','2023-04-05').strftime('%Y-%m-%d').tolist(), 10)
validator_index_data=dict(zip(validator_index_list,date_list))
reward_verification_data=pd.DataFrame()
for validator_index,date in validator_index_data.items():
    res=get_one_day_reward_one_validator(validator_index,date)
    reward_verification_data=pd.concat([reward_verification_data,res])
    reward_verification_data_daily=reward_verification_data.groupby(['date','validatorindex']).agg({'total_reward':'sum','total_attestation_reward':'sum','other_reward':'sum','penalty':'sum'}).reset_index()
    
   

167288 167512


187538 187762
174938 175162
153563 153787
150413 150637
158738 158962
176738 176962
192038 192262
151763 151987
154688 154912


In [4]:
reward_verification_data_daily

,date,validatorindex,total_reward,total_attestation_reward,other_reward,penalty
0,2022-10-01,65002,3186905,3193925,0,7020
1,2022-10-07,8380,3176739,3180505,0,3766
2,2022-10-15,13144,3214672,3214672,0,0
3,2022-10-20,63655,3119228,3143825,0,24597
4,2022-11-07,86746,3140533,3140533,0,0
5,2022-12-15,72574,3028106,3034762,0,6656
6,2023-01-18,51379,3026401,3029936,0,3535
7,2023-01-26,85306,3012925,3012925,0,0
8,2023-03-15,31317,2819157,2835063,0,15906
9,2023-04-04,52444,2809948,2822350,0,12402


## query rewards of these validators in these days in our data

In [5]:
file='/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/eth2_reward_ether/daily_validator_index_reward/total_validator_reward.parquet'
reward_data=pd.read_parquet(file)
reward_data['date']=pd.to_datetime(reward_data['date']).dt.date

In [6]:
df_verification_data=pd.DataFrame()
date_validator=zip(reward_verification_data_daily['date'],reward_verification_data_daily['validatorindex'])
for date,validatorindex in date_validator:
    date=pd.to_datetime(date).date()
    # print(date,validatorindex)
    temp=reward_data[(reward_data['date']==date) & (reward_data['validator_index']==validatorindex)]
    df_verification_data=pd.concat([df_verification_data,temp])

## merge our data and the data from the beaconcha.in

In [7]:
df_verification_data1=pd.merge(df_verification_data,reward_verification_data_daily,left_on=['date','validator_index'],right_on=['date','validatorindex'],how='left')
df_verification_data1['total_attestation_reward']=df_verification_data1['total_attestation_reward']-df_verification_data1['penalty']
df_verification_data1

,date,validator_index,Total reward,Attestation reward,Proposer reward,Sync committee reward,validatorindex,total_reward,total_attestation_reward,other_reward,penalty
0,2022-10-01,65002.0,3186905.0,3186905.0,0.0,0.0,65002,3186905,3186905,0,7020
1,2022-10-07,8380.0,3176739.0,3176739.0,0.0,0.0,8380,3176739,3176739,0,3766
2,2022-10-15,13144.0,3214672.0,3214672.0,0.0,0.0,13144,3214672,3214672,0,0
3,2022-10-20,63655.0,3119228.0,3119228.0,0.0,0.0,63655,3119228,3119228,0,24597
4,2022-11-07,86746.0,3140533.0,3140533.0,0.0,0.0,86746,3140533,3140533,0,0
5,2022-12-15,72574.0,3028106.0,3028106.0,0.0,0.0,72574,3028106,3028106,0,6656
6,2023-01-18,51379.0,3026401.0,3026401.0,0.0,0.0,51379,3026401,3026401,0,3535
7,2023-01-26,85306.0,3012925.0,3012925.0,0.0,0.0,85306,3012925,3012925,0,0
8,2023-03-15,31317.0,2819157.0,2819157.0,0.0,0.0,31317,2819157,2819157,0,15906
9,2023-04-04,52444.0,2809948.0,2809948.0,0.0,0.0,52444,2809948,2809948,0,12402
